In [1]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
import torch.optim as optim
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, cohen_kappa_score

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

SEED = 42
n_splits = 5

# Custom Scorer for Cohen Kappa with Quadratic Weights
def cohen_kappa_quadratic(y_true, y_pred):
    """
    Custom scorer for Cohen's Kappa with quadratic weights.
    Rounds predictions to the nearest integer before calculation.
    """
    y_pred_rounded = y_pred.astype(int)
    y_true_rounded = y_true.astype(int)
    return cohen_kappa_score(y_true_rounded, y_pred_rounded, weights='quadratic')

# Wrapping the custom scorer
kappa_scorer = make_scorer(cohen_kappa_quadratic, greater_is_better=True)

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim*3),
            nn.ReLU(),
            nn.Linear(encoding_dim*3, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim*2),
            nn.ReLU(),
            nn.Linear(input_dim*2, input_dim*3),
            nn.ReLU(),
            nn.Linear(input_dim*3, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    data_tensor = torch.FloatTensor(df_scaled)
    
    input_dim = data_tensor.shape[1]
    autoencoder = AutoEncoder(input_dim, encoding_dim)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters())
    
    for epoch in range(epochs):
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i : i + batch_size]
            optimizer.zero_grad()
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]')
                 
    with torch.no_grad():
        encoded_data = autoencoder.encoder(data_tensor).numpy()
        
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded

def feature_engineering(df):
    season_cols = [col for col in df.columns if 'Season' in col]
    df = df.drop(season_cols, axis=1) 
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    
    return df

def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def map_to_scaled_real(y_pred):
    """
    Map values from 0-100 to a smaller real number with scaling in specific ranges.

    Args:
    - y_pred (array-like): Input values ranging from 0 to 100.

    Returns:
    - Scaled values as per the mapping logic.
    """
    # Initialize the result array
    scaled = np.zeros_like(y_pred, dtype=float)

    # Apply the mapping logic
    scaled = np.where((y_pred >= 0) & (y_pred <= 30), 
                      y_pred / 30,  # Scale 0-30 to 0-1
                      scaled)
    
    scaled = np.where((y_pred > 30) & (y_pred <= 49),
                      1 + (y_pred - 31) / (49 - 31),  # Scale 31-49 to 1-2
                      scaled)
    
    scaled = np.where((y_pred > 50) & (y_pred <= 79),
                      2 + (y_pred - 51) / (79 - 51),  # Scale 51-79 to 2-3
                      scaled)
    
    scaled = np.where((y_pred > 80) & (y_pred <= 100),
                      3 + (y_pred - 81) / (100 - 81),  # Scale 81-100 to 3-4
                      scaled)

    return scaled

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    stats, indexes = zip(*results)
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

# Hyperparameter tuning
def tune_model(regressor, param_grid, X, y):
    pipeline = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('regressor', regressor)])
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring=kappa_scorer,
        cv=5,
        n_jobs=-1,
        verbose=2
    )
    grid_search.fit(X, y)
    print(f"Best Parameters for {regressor.__class__.__name__}: {grid_search.best_params_}")
    return grid_search.best_estimator_

# Load datasets
train = pd.read_csv('/Users/ad53533/Desktop/Applied ML/Project/train.csv')
test = pd.read_csv('/Users/ad53533/Desktop/Applied ML/Project/test.csv')
sample = pd.read_csv('/Users/ad53533/Desktop/Applied ML/Project/sample_submission.csv')

# train_ts = load_time_series("/Users/ad53533/Desktop/Applied ML/Project/series_train.parquet")
# test_ts = load_time_series("/Users/ad53533/Desktop/Applied ML/Project/series_test.parquet")
# 
# df_train = train_ts.drop('id', axis=1)
# df_test = test_ts.drop('id', axis=1)
# 
# train_ts_encoded = perform_autoencoder(df_train, encoding_dim=30, epochs=100, batch_size=32)
# test_ts_encoded = perform_autoencoder(df_test, encoding_dim=30, epochs=100, batch_size=32)

# time_series_cols = train_ts_encoded.columns.tolist()
# train_ts_encoded["id"]=train_ts["id"]
# test_ts_encoded['id']=test_ts["id"]

# train = pd.merge(train, train_ts_encoded, how="left", on='id')
# test = pd.merge(test, test_ts_encoded, how="left", on='id')

imputer = KNNImputer(n_neighbors=5)
numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
imputed_data = imputer.fit_transform(train[numeric_cols])
train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)
train_imputed['sii'] = train_imputed['sii'].round().astype(int)
for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]
        
train = train_imputed

train = feature_engineering(train)
train = train.dropna(thresh=10, axis=0)
test = feature_engineering(test)

train = train.drop('id', axis=1)
test  = test .drop('id', axis=1)   


featuresCols = ['Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-CGAS_Score', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Frame_num',
                'PAQ_A-PAQ_A_Total',
                'PAQ_C-PAQ_C_Total', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T',
                'PreInt_EduHx-computerinternet_hoursday', 'sii', 'BMI_Age','Internet_Hours_Age','BMI_Internet_Hours']


# featuresCols += time_series_cols

train = train.dropna(subset='sii')
train_PCIAT = train['PCIAT-PCIAT_Total']
train = train[featuresCols]
train_sii_mapped = map_to_scaled_real(train_PCIAT)

featuresCols = ['Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-CGAS_Score', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Frame_num',
                'PAQ_A-PAQ_A_Total',
                'PAQ_C-PAQ_C_Total', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T',
                'PreInt_EduHx-computerinternet_hoursday', 'BMI_Age','Internet_Hours_Age','BMI_Internet_Hours']

# featuresCols += time_series_cols
test = test[featuresCols]
if np.any(np.isinf(train)):
    train = train.replace([np.inf, -np.inf], np.nan)

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
        train_sii_mapped_train, train_sii_mapped_test = train_sii_mapped[train_idx], train_sii_mapped[test_idx]
        
        model = clone(model_class)
        model.fit(X_train, train_sii_mapped_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tp_rounded = threshold_Rounder(tpm, KappaOPtimizer.x)

    return tp_rounded


from sklearn.pipeline import Pipeline

# Imputation step: Filling missing values with the median
imputer = KNNImputer(n_neighbors=5)
# Tuning
X = train.drop(['sii'], axis=1)
y = train_sii_mapped
tuned_models = {
    'lgb': tune_model(
        LGBMRegressor(random_state=SEED), 
        {
            'regressor__n_estimators': [500],  # Fewer values for trees
            'regressor__learning_rate': [0.1],  # Common smaller learning rates
            'regressor__max_depth': [7],  # Focus on simpler models
            'regressor__num_leaves': [31],  # Smaller complexity
            'regressor__lambda_l1': [1.0],  # Reduce L1 regularization range
            'regressor__lambda_l2': [0.0],  # Reduce L2 regularization range
        }, 
        X, y
    ),
    'xgb': tune_model(
        XGBRegressor(random_state=SEED), 
        {
            'regressor__n_estimators': [500],  # Same reduced range for consistency
            'regressor__learning_rate': [0.05],
            'regressor__max_depth': [6],
            'regressor__subsample': [0.8],  # Limited to effective ranges
            'regressor__colsample_bytree': [1.0],
        }, 
        X, y
    ),
    'cat': tune_model(
        CatBoostRegressor(random_state=SEED, silent=True), 
        {
            'regressor__iterations': [500],  # Fewer iterations
            'regressor__learning_rate': [0.1],
            'regressor__depth': [8],
            'regressor__l2_leaf_reg': [1],  # Simplified L2 range
        }, 
        X, y
    ),
}


# Ensemble with tuned models
ensemble = VotingRegressor(estimators=[
    ('lgb', tuned_models['lgb']),
    ('xgb', tuned_models['xgb']),
    ('cat', tuned_models['cat']),
])


# Train and predict
predictions = TrainML(ensemble, test)

# Save predictions
# sample_submission = pd.read_csv('sample_submission.csv')
sample['sii'] = predictions
sample.to_csv('submission.csv', index=False)

Training Folds: 100%|██████████| 5/5 [00:14<00:00,  2.95s/it]

Mean Train QWK --> 0.8800
Mean Validation QWK ---> 0.4993
----> || Optimized QWK SCORE ::  0.546


In [2]:
sample

,id,sii
0,00008ff9,0
1,000fd460,0
2,00105258,1
3,00115b9f,0
4,0016bb22,1
5,001f3379,1
6,0038ba98,1
7,0068a485,0
8,0069fbed,1
9,0083e397,1
